### Imports

In [10]:
import os
import numpy as np
import pandas as pd

### Inputs

In [11]:
fusion_output_df = pd.read_csv("../outputs/fusion_output.csv")
fusion_output_class = fusion_output_df['class'].values
fusion_output_confidence = fusion_output_df['confidence'].values

In [15]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

### Kalman Filter

In [17]:
class KalmanFilter:
    def __init__(self, process_variance=1e-3, measurement_variance=1e-2, initial_estimate=0.5):
        self.x = initial_estimate       
        self.P = 1.0                    
        self.Q = process_variance       
        self.R = measurement_variance   

    def update(self, measurement):
        self.P += self.Q
        K = self.P / (self.P + self.R)
        self.x += K * (measurement - self.x)
        self.P *= (1 - K)
        return self.x

In [23]:
def apply_kalman(confidences):
    kf = KalmanFilter()
    smoothed = []
    for z in confidences:
        smoothed.append(kf.update(z))
    return np.array(smoothed)

### Apply

In [24]:
conf = apply_kalman(fusion_output_confidence)

In [25]:
tau = 0.8
cls = (conf >= tau).astype(int)

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

In [28]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[cls[i]]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-55    2.0   17     0.30   1        Bom      0.85      
-85    3.8   30     0.10   0        Ruim     0.74      
-35    1.2   5      0.05   0        Ruim     0.79      
-88    9.5   95     0.90   1        Bom      0.82      
-40    8.2   85     0.80   1        Bom      0.84      
-87    1.5   8      0.15   1        Bom      0.83      
-62    2.8   32     0.28   1        Bom      0.84      
-58    3.2   28     0.32   1        Bom      0.84      
-52    2.1   20     0.25   1        Bom      0.86      
-48    2.5   25     0.22   1        Bom      0.86      
-50    1.8   22     0.28   1        Bom      0.87      
-75    5.2   65     0.60   1        Bom      0.87      
-78    4.8   70     0.65   1        Bom      0.88      
-72    5.5   60     0.58   1        Bom      0.88      
-45    4.5   15     0.70   1        Bom      0.83      
-70    1.9   45     0.20   1        Bom      0.8

### Save

In [29]:
kalman_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
kalman_output.to_csv("../outputs/kalman_output.csv", index=False)

print("Filtro de Kalman salvo em ../outputs/kalman_output.csv")

Filtro de Kalman salvo em ../outputs/kalman_output.csv
